In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-8c74f297-f026-484d-93d3-c048f8e2b386"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-f5823dc5-753d-4db7-a052-0bf0cfe9c3cf"
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

In [2]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
from core.interpolator import Interpolator

In [3]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [4]:
application = Application(client, compiler)
interpolator = Interpolator(".")

In [5]:
application_description = """
Bot that helps me planning my YouTube content and posting schedule.
""".strip()

In [6]:
application_description = "Bot that greets user by name and returns personalized greeting."

In [7]:
my_bot = application.create_bot(application_description, bot_id="handler_tests")

Skipping Initial Description Refinement
Compiling TypeSpec...
Compiling Typescript Schema Definitions...
Compiling Drizzle...
Compiling Handler Tests...
Compiling Handlers...


In [8]:
interpolator.bake(my_bot, "../../bot_result") # next to repo root

In [9]:
my_bot.__dict__

{'refined_description': RefineOut(refined_description='Bot that greets user by name and returns personalized greeting.', error_output=None),
 'typespec': TypespecOut(reasoning='I expect user to send messages like "Hello, my name is John" or "Hi, I\'m Jane".\nLLM can extract the name from the message and pass it to the greet function.\n"Hello, my name is John" -> greet({name: "John", language: "en"})\nThe bot should support multiple languages and store user preferences.\n- greet(options: GreetRequest): void;\n- setLanguage(options: LanguagePreference): void;', typespec_definitions='model GreetRequest {\n    name: string;\n    language: string;\n}\n\nmodel LanguagePreference {\n    language: string;\n    isDefault: boolean;\n}\n\ninterface GreetingBot {\n    @llm_func("Generate personalized greeting for user")\n    greet(options: GreetRequest): string;\n\n    @llm_func("Set user\'s preferred language for greetings")\n    setLanguage(options: LanguagePreference): void;\n}', llm_functions=